In [686]:
class Parser():
    def __init__(self, file):
        with open(file) as f:
            self.commands = f.readlines()
            for i in range(len(self.commands)):
            
                comment = self.commands[i].find("//")
                if comment != -1:
                    self.commands[i] = self.commands[i][:comment]
                self.commands[i] = self.commands[i].replace(" ", "")
                self.commands[i] = self.commands[i].replace("\n", "")
                
            self.commands = [i for i in self.commands if i  != ""]
            self.current = self.commands[0]
            self.num_com = len(self.commands)
            self.count = 0
            self.eq_pos = self.current.find("=")
            self.sc_pos = self.current.find(";")
            self.rom_num = 0
    def hasMoreCommands(self):
            if self.count + 1 >= self.num_com:
                return False
            else:
                return True

    def advance(self):
        if self.hasMoreCommands():
            self.count += 1
            self.current = self.commands[self.count]
            self.eq_pos = self.current.find("=")
            self.sc_pos = self.current.find(";")


    def commandType(self):
        if self.current[0] == "(":
            return "L"
        elif self.current[0] == "@":
            return "C"
        else:
            return "A"


    def dest(self):
        if self.eq_pos != -1:
            return self.current[:self.eq_pos]
        else:
            return None

    def comp(self):
        if self.eq_pos == -1:
            return self.current[:self.sc_pos]
        elif self.sc_pos == -1:
            return self.current[self.eq_pos+1:]
        else:
            return self.current[self.eq_pos+1:self.sc_pos]


    def jump(self):

        if self.sc_pos != -1:
            return self.current[self.sc_pos+1:]
        else:
            return None
        
    def c_symbol(self):
        try:
             return  int(self.current[1:])
        except  ValueError:
            return  self.current[1:]
    
    def l_symbol(self):
        return self.current[1:-1]
    
    
    
    

In [714]:
class Code():
    def dest(self, p):
        d = p.dest()
        if d == None:
            return "000"
        elif d == "M":
            return "001"
        elif d == "D":
            return "010"
        elif d == "MD" or d == "DM":
            return "011"
        elif d == "A":
            return "100"
        elif d == "AM" or d == "MA":
            return "101"
        elif d == "AD" or d == "DA":
            return "110"
        elif d == "ADM" or d == "AMD" or d == "DAM" or d == "DMA" or d == "MDA" or d == "MAD" :
            return "111"

    def comp(self, p):
        c = p.comp()

        if c == "0":
            return "0101010"
        elif c == "1":
            return "0111111"
        elif c == "-1":
            return "0111010"
        elif c == "D":
            return "0001100"
        elif c == "A":
            return "0110000"
        elif c == "!D":
            return "0001101"
        elif c == "!A":
            return "0110001"
        elif c == "-D":
            return "0001111"
        elif c == "-A":
            return "0110011"
        elif c == "D+1":
            return "0011111"
        elif c == "A+1":
            return "0110111"
        elif c == "D-1":
            return "0001110"
        elif c == "A-1":
            return "0110010"
        elif c == "D+A":
            return "0000010"
        elif c == "D-A":
            return "0010011"
        elif c == "A-D":
            return "0000111"
        elif c == "D&A":
            return "0000000"
        elif c == "D|A":
            return "0010101"
        elif c == "M":
            return "1110000"
        elif c == "!M":
            return "1110001"
        elif c == "-M":
            return "1110011"
        elif c == "M+1":
            return "1110111"
        elif c == "M-1":
            return "1110010"
        elif c == "D+M":
            return "1000010"
        elif c == "D-M":
            return "1010011"
        elif c == "M-D":
            return "1000111"
        elif c == "D&M":
            return "1000000"
        elif c == "D|M":
            return "1010101"

    def jump(self, p):
        j = p.jump()
        if j == None:
            return "000"
        elif j == "JGT":
            return "001"
        elif j == "JEQ":
            return "010"
        elif j == "JGE":
            return "011"
        elif j == "JLT":
            return "100"
        elif j == "JNE":
            return "101"
        elif j == "JLE":
            return "110"
        elif j == "JMP":
            return "111"
        
    def c_bin(self, p, st):
        if type(p.c_symbol()) is int:
            return format(p.c_symbol(), "015b")
        else:
            return format(st.s_table[p.c_symbol()], "015b")
        


In [720]:
class STable():
    def __init__(self):
        self.s_table = {"SP":0, "LCL":1, "ARG":2, "THIS":3, "THAT":4,
                        "R0":0, "R1":1, "R2":2, "R3":3, "R4":4, "R5":5, "R6":6,
                        "R7": 7,"R8":8, "R9":9, "R10":10, "R11":11, "R12":12,
                        "R13": 13,"R14":14, "R15":15, "SCREEN": 16384, "KBD": 24576}
        self.c_num = 16
        self.l_num = 0
        
    def contains(self, symbol):
        if list(self.s_table.keys()).count(symbol) == 1:
            return True
        else:
            return False

    def addEntry(self, symbol, order):
        if not self.contains(symbol):
            if order == "C":
                self.s_table[symbol] = self.c_num
                self.c_num += 1
            elif order == "L":
                
                self.s_table[symbol] = self.l_num
                
        
    def getAddress(self, symbol):
        return self.s_table[symbol]

In [721]:
def assembler(asm_file, hack_file):
    p = Parser(asm_file)
    c = Code()
    st = STable()

    ## labeling & symbol translation
    while True:
        if p.commandType() == "A": 
            st.l_num += 1

        elif p.commandType() == "L":
            st.addEntry(p.l_symbol(), "L")

        elif p.commandType() == "C":
            st.l_num += 1 

        if not p.hasMoreCommands():
            p.count = 0
            p.current = p.commands[0]
            break
        p.advance()
        
    while True:
        if p.commandType() == "A": 
            pass

        elif p.commandType() == "L":
            pass

        elif p.commandType() == "C":
            if type(p.c_symbol()) is int: 
                pass
            else:
                if  not st.contains(p.c_symbol()):
                    st.addEntry(p.c_symbol(), "C")
                 

        if not p.hasMoreCommands():
            p.count = 0
            p.current = p.commands[0]
            break
        p.advance()
    print(st.s_table)
    ## encoding
    binary_codes = []
    while True:
        print(p.current)
        if p.commandType() == "A": 
            
            s = "111" + c.comp(p) + c.dest(p) + c.jump(p)
            binary_codes.append(s)
        elif p.commandType() == "C":
            s = "0" + c.c_bin(p, st)
            binary_codes.append(s)
        
        
        if not p.hasMoreCommands():
            p.count = 0
            p.current = p.commands[0]
            break

        p.advance()

    
    with open(hack_file, mode='w') as f:
        f.write('\n'.join(binary_codes))


In [722]:
assembler("/Users/manako/nand2tetris/projects/06/add/Add.asm", "/Users/manako/nand2tetris/projects/06/add/Add2.hack")
assembler("/Users/manako/nand2tetris/projects/06/max/Max.asm", "/Users/manako/nand2tetris/projects/06/max/Max2.hack")
assembler("/Users/manako/nand2tetris/projects/06/max/MaxL.asm", "/Users/manako/nand2tetris/projects/06/max/MaxL2.hack")
assembler("/Users/manako/nand2tetris/projects/06/rect/Rect.asm", "/Users/manako/nand2tetris/projects/06/rect/Rect2.hack")
assembler("/Users/manako/nand2tetris/projects/06/rect/RectL.asm", "/Users/manako/nand2tetris/projects/06/rect/RectL2.hack")
assembler("/Users/manako/nand2tetris/projects/06/pong/Pong.asm", "/Users/manako/nand2tetris/projects/06/pong/Pong2.hack")
assembler("/Users/manako/nand2tetris/projects/06/pong/PongL.asm", "/Users/manako/nand2tetris/projects/06/pong/PongL2.hack")

{'SP': 0, 'LCL': 1, 'ARG': 2, 'THIS': 3, 'THAT': 4, 'R0': 0, 'R1': 1, 'R2': 2, 'R3': 3, 'R4': 4, 'R5': 5, 'R6': 6, 'R7': 7, 'R8': 8, 'R9': 9, 'R10': 10, 'R11': 11, 'R12': 12, 'R13': 13, 'R14': 14, 'R15': 15, 'SCREEN': 16384, 'KBD': 24576}
@2
D=A
@3
D=D+A
@0
M=D
{'SP': 0, 'LCL': 1, 'ARG': 2, 'THIS': 3, 'THAT': 4, 'R0': 0, 'R1': 1, 'R2': 2, 'R3': 3, 'R4': 4, 'R5': 5, 'R6': 6, 'R7': 7, 'R8': 8, 'R9': 9, 'R10': 10, 'R11': 11, 'R12': 12, 'R13': 13, 'R14': 14, 'R15': 15, 'SCREEN': 16384, 'KBD': 24576, 'OUTPUT_FIRST': 10, 'OUTPUT_D': 12, 'INFINITE_LOOP': 14}
@R0
D=M
@R1
D=D-M
@OUTPUT_FIRST
D;JGT
@R1
D=M
@OUTPUT_D
0;JMP
(OUTPUT_FIRST)
@R0
D=M
(OUTPUT_D)
@R2
M=D
(INFINITE_LOOP)
@INFINITE_LOOP
0;JMP
{'SP': 0, 'LCL': 1, 'ARG': 2, 'THIS': 3, 'THAT': 4, 'R0': 0, 'R1': 1, 'R2': 2, 'R3': 3, 'R4': 4, 'R5': 5, 'R6': 6, 'R7': 7, 'R8': 8, 'R9': 9, 'R10': 10, 'R11': 11, 'R12': 12, 'R13': 13, 'R14': 14, 'R15': 15, 'SCREEN': 16384, 'KBD': 24576}
@0
D=M
@1
D=D-M
@10
D;JGT
@1
D=M
@12
0;JMP
@0
D=M
@2
M=D
@14
0

A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@1
D=A
@R13
M=D
@math.abs
D=A
@R14
M=D
@RET_ADDRESS_CALL10
D=A
@95
0;JMP
(RET_ADDRESS_CALL10)
@SP
AM=M-1
D=M
@LCL
A=M+1
M=D
@LCL
A=M
D=M
@SP
AM=M+1
A=A-1
M=D
@LCL
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@RET_ADDRESS_LT0
D=A
@38
0;JMP
(RET_ADDRESS_LT0)
@THIS
D=M
@7
D=D+A
@R13
M=D
@SP
AM=M-1
D=M
@R13
A=M
M=D
@THIS
D=M
@7
A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@SP
AM=M-1
D=M
@ball.setdestination$if_true0
D;JNE
@ball.setdestination$if_false0
0;JMP
(ball.setdestination$if_true0)
@LCL
A=M
D=M
@SP
AM=M+1
A=A-1
M=D
@SP
AM=M-1
D=M
@LCL
A=M+1
A=A+1
M=D
@LCL
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@SP
AM=M-1
D=M
@LCL
A=M
M=D
@LCL
A=M+1
A=A+1
D=M
@SP
AM=M+1
A=A-1
M=D
@SP
AM=M-1
D=M
@LCL
A=M+1
M=D
@THIS
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@ARG
A=M+1
A=A+1
D=M
@SP
AM=M+1
A=A-1
M=D
@RET_ADDRESS_LT1
D=A
@38
0;JMP
(RET_ADDRESS_LT1)
@THIS
D=M
@8
D=D+A
@R13
M=D
@SP
AM=M-1
D=M
@R13
A=M
M=D
@THIS
A=M
D=M
@SP
AM=M+1
A=A-1
M=D
@ARG
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@RET_ADDRESS_LT2
D=A
@38
0;JMP
(RET_ADDRESS_LT

D=M
@ball.bounce$if_true3
D;JNE
@ball.bounce$if_false3
0;JMP
(ball.bounce$if_true3)
@SP
M=M+1
A=M-1
M=0
@SP
AM=M-1
D=M
@LCL
A=M
M=D
@LCL
D=M
@3
A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@50
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@math.multiply
D=A
@R14
M=D
@RET_ADDRESS_CALL21
D=A
@95
0;JMP
(RET_ADDRESS_CALL21)
@LCL
A=M+1
A=A+1
D=M
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@math.divide
D=A
@R14
M=D
@RET_ADDRESS_CALL22
D=A
@95
0;JMP
(RET_ADDRESS_CALL22)
@SP
AM=M-1
D=M
@LCL
A=M+1
M=D
@THIS
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@LCL
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@LCL
D=M
@4
A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@math.multiply
D=A
@R14
M=D
@RET_ADDRESS_CALL23
D=A
@95
0;JMP
(RET_ADDRESS_CALL23)
@SP
AM=M-1
D=M
A=A-1
M=D+M
@SP
AM=M-1
D=M
@LCL
A=M+1
M=D
@ball.bounce$if_end3
0;JMP
(ball.bounce$if_false3)
@THIS
D=M
@14
A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@3
D=A
@SP
AM=M+1
A=A-1
M=D
@RET_ADDRESS_EQ5
D=A
@6
0;JMP
(RET_ADDRESS_EQ5)
@SP
AM=M-1
D=M
@ball.bounce$if_true4
D;JNE
@ball.bounce$if_false4
0;JMP
(ball.bounce

D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL86
D=A
@95
0;JMP
(RET_ADDRESS_CALL86)
@101
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL87
D=A
@95
0;JMP
(RET_ADDRESS_CALL87)
@32
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL88
D=A
@95
0;JMP
(RET_ADDRESS_CALL88)
@79
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL89
D=A
@95
0;JMP
(RET_ADDRESS_CALL89)
@118
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL90
D=A
@95
0;JMP
(RET_ADDRESS_CALL90)
@101
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL91
D=A
@95
0;JMP
(RET_ADDRESS_CALL91)
@114
D=A
@SP
AM=M+1
A=A-1
M=D
@2
D=A
@R13
M=D
@string.appendchar
D=A
@R14
M=D
@RET_ADDRESS_CALL92
D=A
@95
0;JMP
(RET_ADDRESS_CALL92)
@1
D=A
@R13
M=D
@output.printstring
D=A
@R14
M=D
@RET_ADDRESS_CALL93
D=A
@95
0;JMP
(RET_ADDRESS_CALL93)


M=D
@string.newline
D=A
@R14
M=D
@RET_ADDRESS_CALL114
D=A
@95
0;JMP
(RET_ADDRESS_CALL114)
@SP
AM=M-1
D=M
@LCL
A=M+1
M=D
@0
D=A
@R13
M=D
@string.backspace
D=A
@R14
M=D
@RET_ADDRESS_CALL115
D=A
@95
0;JMP
(RET_ADDRESS_CALL115)
@SP
AM=M-1
D=M
@LCL
A=M+1
A=A+1
M=D
(keyboard.readline$while_exp0)
@LCL
D=M
@4
A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@SP
A=M-1
M=!M
@SP
A=M-1
M=!M
@SP
AM=M-1
D=M
@keyboard.readline$while_end0
D;JNE
@0
D=A
@R13
M=D
@keyboard.readchar
D=A
@R14
M=D
@RET_ADDRESS_CALL116
D=A
@95
0;JMP
(RET_ADDRESS_CALL116)
@SP
AM=M-1
D=M
@LCL
A=M
M=D
@LCL
A=M
D=M
@SP
AM=M+1
A=A-1
M=D
@LCL
A=M+1
D=M
@SP
AM=M+1
A=A-1
M=D
@RET_ADDRESS_EQ15
D=A
@6
0;JMP
(RET_ADDRESS_EQ15)
@SP
AM=M-1
D=M
@LCL
A=M+1
A=A+1
A=A+1
A=A+1
M=D
@LCL
D=M
@4
A=D+A
D=M
@SP
AM=M+1
A=A-1
M=D
@SP
A=M-1
M=!M
@SP
AM=M-1
D=M
@keyboard.readline$if_true0
D;JNE
@keyboard.readline$if_false0
0;JMP
(keyboard.readline$if_true0)
@LCL
A=M
D=M
@SP
AM=M+1
A=A-1
M=D
@LCL
A=M+1
A=A+1
D=M
@SP
AM=M+1
A=A-1
M=D
@RET_ADDRESS_EQ16
D=A
@6
0;JMP
(RET_A

...

M=0
@0
M=M+1
A=M-1
M=0
@51
D=A
@0
AM=M+1
A=A-1
M=D
@51
D=A
@0
AM=M+1
A=A-1
M=D
@51
D=A
@0
AM=M+1
A=A-1
M=D
@51
D=A
@0
AM=M+1
A=A-1
M=D
@30
D=A
@0
AM=M+1
A=A-1
M=D
@12
D=A
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@12
D=A
@13
M=D
@17843
D=A
@14
M=D
@17186
D=A
@95
0;JMP
@0
AM=M-1
D=M
@5
M=D
@119
D=A
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@51
D=A
@0
AM=M+1
A=A-1
M=D
@51
D=A
@0
AM=M+1
A=A-1
M=D
@51
D=A
@0
AM=M+1
A=A-1
M=D
@63
D=A
@0
AM=M+1
A=A-1
M=D
@63
D=A
@0
AM=M+1
A=A-1
M=D
@18
D=A
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@12
D=A
@13
M=D
@17843
D=A
@14
M=D
@17265
D=A
@95
0;JMP
@0
AM=M-1
D=M
@5
M=D
@120
D=A
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@51
D=A
@0
AM=M+1
A=A-1
M=D
@30
D=A
@0
AM=M+1
A=A-1
M=D
@12
D=A
@0
AM=M+1
A=A-1
M=D
@12
D=A
@0
AM=M+1
A=A-1
M=D
@30
D=A
@0
AM=M+1
A=A-1
M=D
@51
D=A
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@0
M=M+1
A=M-1
M=0
@12
D=A
@13
M=D
@17843
D=A
@14
M=

M=D
@0
AM=M-1
D=M
A=A-1
M=D+M
@0
AM=M-1
D=M
@4
M=D
@4
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@255
D=A
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=D&M
@0
AM=M-1
D=M
@1
A=M+1
A=A+1
A=A+1
M=D
@1
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@20
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=D+M
@1
A=M+1
D=M
@0
AM=M+1
A=A-1
M=D
@1
A=M+1
A=A+1
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=D+M
@0
AM=M-1
D=M
@4
M=D
@4
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@1
D=M
@3
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=D|M
@0
AM=M-1
D=M
@5
M=D
@0
AM=M-1
D=M
@4
M=D
@5
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
@4
A=M
M=D
@1
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@32
D=A
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=D+M
@0
AM=M-1
D=M
@1
A=M
M=D
@1
A=M+1
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=1
@0
AM=M-1
D=M
A=A-1
M=D+M
@0
AM=M-1
D=M
@1
A=M+1
M=D
@19068
0;JMP
@0
M=M+1
A=M-1
M=0
@54
0;JMP
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@19365
D=A
@38
0;JMP
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@22
D=A
@0
AM=M+1
A=A-1
M=D
@19382
D=A
@22
0;JMP
@0
AM=M-1
D=M
A=A-1
M

@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=M-D
@1
D=A
@13
M=D
@6774
D=A
@14
M=D
@21442
D=A
@95
0;JMP
@0
AM=M-1
D=M
@1
A=M+1
A=A+1
A=A+1
M=D
@2
D=M
@3
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@2
A=M+1
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
A=A-1
M=M-D
@1
D=A
@13
M=D
@6774
D=A
@14
M=D
@21483
D=A
@95
0;JMP
@0
AM=M-1
D=M
@1
A=M+1
A=A+1
M=D
@1
D=M
@3
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@1
A=M+1
A=A+1
D=M
@0
AM=M+1
A=A-1
M=D
@21511
D=A
@38
0;JMP
@0
AM=M-1
D=M
@1
A=M+1
A=A+1
A=A+1
A=A+1
A=A+1
A=A+1
M=D
@1
D=M
@6
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@2
D=M
@3
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@2
A=M+1
D=M
@0
AM=M+1
A=A-1
M=D
@21551
D=A
@38
0;JMP
@0
AM=M-1
D=M
A=A-1
M=D&M
@1
D=M
@6
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@0
A=M-1
M=!M
@2
A=M+1
A=A+1
D=M
@0
AM=M+1
A=A-1
M=D
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@21587
D=A
@38
0;JMP
@0
AM=M-1
D=M
A=A-1
M=D&M
@0
AM=M-1
D=M
A=A-1
M=D|M
@0
AM=M-1
D=M
@21604
D;JNE
@21698
0;JMP
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
@1
A=M+1
A=A+1
A=A+1
A=A+1
M=D
@2
A=M+1
A=A+1
D=M
@0
AM=M+1
A=A-1

M=D
@8406
D=A
@14
M=D
@23287
D=A
@95
0;JMP
@1
D=M
@7
D=D+A
@13
M=D
@0
AM=M-1
D=M
@13
A=M
M=D
@2
A=M+1
D=M
@0
AM=M+1
A=A-1
M=D
@2
A=M+1
A=A+1
D=M
@0
AM=M+1
A=A-1
M=D
@2
D=A
@13
M=D
@8359
D=A
@14
M=D
@23326
D=A
@95
0;JMP
@1
D=M
@8
D=D+A
@13
M=D
@0
AM=M-1
D=M
@13
A=M
M=D
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=1
@0
A=M-1
D=!M
M=D+1
@23357
D=A
@22
0;JMP
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@256
D=A
@0
AM=M+1
A=A-1
M=D
@23374
D=A
@38
0;JMP
@0
AM=M-1
D=M
A=A-1
M=D&M
@1
D=M
@7
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@512
D=A
@0
AM=M+1
A=A-1
M=D
@23398
D=A
@38
0;JMP
@0
AM=M-1
D=M
A=A-1
M=D&M
@1
D=M
@8
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=1
@0
A=M-1
D=!M
M=D+1
@23424
D=A
@22
0;JMP
@0
AM=M-1
D=M
A=A-1
M=D&M
@0
AM=M-1
D=M
@23436
D;JNE
@24145
0;JMP
@1
D=M
@7
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@2
D=A
@13
M=D
@8359
D=A
@14
M=D
@23461
D=A
@95
0;JMP
@1
D=M
@7
D=D+A
@13
M=D
@0
AM=M-1
D=M
@13
A=M
M=D
@1
D=M
@8
A=D+A
D=M
@0
AM=M+1
A=A-1
M=D
@511
D=A
@0
AM=M+1
A=A-1
M=D
@2
D=A
@13
M=D
@840

M=D
@24151
D=A
@14
M=D
@25092
D=A
@95
0;JMP
@0
AM=M-1
D=M
@5
M=D
@24833
0;JMP
@0
M=M+1
A=M-1
M=0
@54
0;JMP
@3
D=A
@0
AM=M+1
A=A-1
M=D
@1
D=A
@13
M=D
@8643
D=A
@14
M=D
@25123
D=A
@95
0;JMP
@0
AM=M-1
D=M
@3
M=D
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@25143
D=A
@38
0;JMP
@0
AM=M-1
D=M
@25150
D;JNE
@25173
0;JMP
@14
D=A
@0
AM=M+1
A=A-1
M=D
@1
D=A
@13
M=D
@27353
D=A
@14
M=D
@25168
D=A
@95
0;JMP
@0
AM=M-1
D=M
@5
M=D
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@25188
D=A
@22
0;JMP
@0
AM=M-1
D=M
@25195
D;JNE
@25220
0;JMP
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@1
D=A
@13
M=D
@5808
D=A
@14
M=D
@25214
D=A
@95
0;JMP
@0
AM=M-1
D=M
@3
A=M+1
M=D
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
@3
A=M
M=D
@0
M=M+1
A=M-1
M=0
@0
AM=M-1
D=M
@3
A=M+1
A=A+1
M=D
@3
D=M
@0
AM=M+1
A=A-1
M=D
@54
0;JMP
@2
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
AM=M-1
D=M
@3
M=D
@3
A=M
D=M
@0
AM=M+1
A=A-1
M=D
@0
M=M+1
A=M-1
M=0
@25279
D=A
@22
0;JMP
@0
AM=M-1
D=M
@25286
D;JNE
@25310
0;JMP
@3
A=M+1
D=M
@0
AM=M+1
A=A-1
M=D
@1
D=A
@13
M